MRP计算v(s)的方法1：动态规划迭代法

In [1]:
"""这是MRP迭代过程"""
 
 
# 迭代
def next_v(_lambda, r_list, old_v_list, weight_list):
    new_v_list = []
    for j in range(len(old_v_list)):
        if j != len(old_v_list) - 1:
            j_sum = .0
            # 与当前状态相接的后续状态的值函数相应的状态转移概率的求和的折扣
            for k in range(len(weight_list[j])):
                j_sum += weight_list[j][k][0] * old_v_list[weight_list[j][k][1]]
 
            # 当前状态在下一阶段的报酬
            new_v_list.append(r_list[j] + _lambda * j_sum)
 
    # Sleep状态无后续状态，故直接赋值0
    new_v_list.append(0.0)
    return new_v_list
 
 
if __name__ == '__main__':
    # γ
    my_lambda = 1
 
    # 报酬顺序：Class 1、Class 2、Class 3、Pass、Pub、Facebook、Sleep，分别对应0, 1, 2, 3, 4, 5, 6
    # 后续顺序皆与此相同
    my_r_list = [-2., -2., -2., 10., 1., -1., 0.]
 
    # 初始化值函数
    my_old_v_list = [0, 0, 0, 0, 0, 0, 0]
 
    # 状态转移概率（这里没有用概率矩阵，方法有点笨，读者可以用矩阵来表示）
    # 这里以[[0.5, 1], [0.5, 5]]为例解释一下，该列表记录Class 1的状态：
    # [0.5, 1]表示以0.5的概率转移到Class 2
    # [0.5, 5]表示以0.5的概率转移到Facebook
    my_weight_list = [[[0.5, 1], [0.5, 5]],
                      [[0.8, 2], [0.2, 6]],
                      [[0.6, 3], [0.4, 4]],
                      [[1, 6]],
                      [[0.2, 0], [0.4, 1], [0.4, 2]],
                      [[0.1, 0], [0.9, 5]],
                      [[0, 0]]]
 
    my_new_v_list = []
    # 指定迭代次数
    for i in range(100):
        my_new_v_list = next_v(my_lambda, my_r_list, my_old_v_list, my_weight_list)
 
        # 用新生成的值函数列表替换旧的值函数列表
        my_old_v_list = my_new_v_list
 
    print(my_new_v_list)

[-12.418287702397645, 1.4703089405374907, 4.3371337110593915, 10.0, 0.8410368812854547, -22.318009521720207, 0.0]


MRP计算v(s)的方法2：矩阵解析法

In [3]:
import numpy as np
 
 
# γ
_lambda = 1
 
# 状态转移概率矩阵
p = [[0, 0.5, 0, 0, 0, 0.5, 0],
     [0, 0, 0.8, 0, 0, 0, 0.2],
     [0, 0, 0, 0.6, 0.4, 0, 0],
     [0, 0, 0, 0, 0, 0, 1],
     [0.2, 0.4, 0.4, 0, 0, 0, 0],
     [0.1, 0, 0, 0, 0, 0.9, 0],
     [0, 0, 0, 0, 0, 0, 0]]
# 报酬矩阵
r = [[-2],
     [-2],
     [-2],
     [10],
     [1],
     [-1],
     [0]]
# 单位矩阵
i = [[1, 0, 0, 0, 0, 0, 0],
     [0, 1, 0, 0, 0, 0, 0],
     [0, 0, 1, 0, 0, 0, 0],
     [0, 0, 0, 1, 0, 0, 0],
     [0, 0, 0, 0, 1, 0, 0],
     [0, 0, 0, 0, 0, 1, 0],
     [0, 0, 0, 0, 0, 0, 1]]
 
p_mat = np.matrix(p)
r_mat = np.matrix(r)
i_mat = np.matrix(i)
 
# Bellman等式的矩阵形式
v_mat = (i_mat - _lambda * p_mat).I * r_mat
# v_mat = np.dot(np.linalg.inv(i_mat - p_mat), r_mat)
 
print(v_mat)

[[-12.54320988]
 [  1.45679012]
 [  4.32098765]
 [ 10.        ]
 [  0.80246914]
 [-22.54320988]
 [  0.        ]]


MDP计算Q(s,a)的方法1:基于动态规划算法的Bellman等式迭代

In [5]:
"""这是MDP迭代过程"""
 
 
def next_v(pi, r_list, old_v_list, weight_list):
    new_v_list = []
    for j in range(len(old_v_list)):
        if j != len(old_v_list) - 1:
            j_sum = .0
            for k in range(len(weight_list[j])):
                if type(weight_list[j][k][0]) is not list:
                    j_sum += pi * (r_list[weight_list[j][k][1]] + old_v_list[weight_list[j][k][0]])
                else:
                    m_sum = .0
                    for m in range(len(weight_list[j][k][0])):
                        m_sum += old_v_list[weight_list[j][k][0][m][0]] * weight_list[j][k][0][m][1]
                    j_sum += pi * (r_list[weight_list[j][k][1]] + m_sum)
            new_v_list.append(j_sum)
 
    new_v_list.append(0.0)
    return new_v_list
 
 
if __name__ == '__main__':
    my_pi = 0.5
    # 报酬顺序：study pass pub facebook quit sleep
    my_r_list = [-2., 10., 1., -1., 0., 0.]
    my_old_v_list = [0, 0, 0, 0, 0]
    my_weight_list = [[[1, 0], [3, 3]],
                      [[2, 0], [4, 5]],
                      [[[[0, 0.2], [1, 0.4], [2, 0.4]], 2], [4, 1]],
                      [[0, 4], [3, 3]],
                      []]
 
    my_new_v_list = []
    # my_new_v_list = next_v(my_pi, my_r_list, my_old_v_list, my_weight_list)
    for i in range(100):
        my_new_v_list = next_v(my_pi, my_r_list, my_old_v_list, my_weight_list)
        my_old_v_list = my_new_v_list
 
    print(my_new_v_list)

[-1.3076923099959044, 2.6923076920317515, 7.384615384159404, -2.3076923112229597, 0.0]


MDP计算Q(s,a)的方法2:Bellman等式的矩阵解析解

In [8]:
import numpy as np
 
 
# π、γ
_pi = 0.5
_lambda = 1
 
p = [[0, _pi, 0, _pi, 0],
     [0, 0, _pi, 0, _pi],
     [_pi*0.2, _pi*0.4, _pi*0.4, 0, _pi],
     [_pi, 0, 0, _pi, 0],
     [0, 0, 0, 0, 0]]
r = [[_pi*-2 + _pi*-1],
     [_pi*-2 + _pi*0],
     [_pi*1 + _pi*10],
     [_pi*0 + _pi*-1],
     [0]]
i = [[1, 0, 0, 0, 0],
     [0, 1, 0, 0, 0],
     [0, 0, 1, 0, 0],
     [0, 0, 0, 1, 0],
     [0, 0, 0, 0, 1]]
 
p_mat = np.matrix(p)
r_mat = np.matrix(r)
i_mat = np.matrix(i)
 
v_mat = (i_mat - _lambda * p_mat).I * r_mat
# v_mat = np.dot(np.linalg.inv(i_mat - p_mat), r_mat)
 
print(v_mat)

[[-1.30769231]
 [ 2.69230769]
 [ 7.38461538]
 [-2.30769231]
 [ 0.        ]]


求最优值函数

In [9]:
"""这是OVF迭代过程"""
 
 
def next_v(pi, r_list, old_v_list, weight_list):
    new_v_list = []
    for j in range(len(old_v_list)):
        if j != len(old_v_list) - 1:
            max_list = []
            for k in range(len(weight_list[j])):
                if type(weight_list[j][k][0]) is not list:
                    max_list.append(pi * (r_list[weight_list[j][k][1]] + old_v_list[weight_list[j][k][0]]))
                else:
                    m_sum = .0
                    for m in range(len(weight_list[j][k][0])):
                        m_sum += old_v_list[weight_list[j][k][0][m][0]] * weight_list[j][k][0][m][1]
                    max_list.append(pi * (r_list[weight_list[j][k][1]] + m_sum))
            new_v_list.append(max(max_list))
 
    new_v_list.append(0.0)
    return new_v_list
 
 
if __name__ == '__main__':
    my_pi = 1
    # study pass pub facebook quit sleep
    my_r_list = [-2., 10., 1., -1., 0., 0.]
    my_old_v_list = [0, 0, 0, 0, 0]
    my_weight_list = [[[1, 0], [3, 3]],
                      [[2, 0], [4, 5]],
                      [[[[0, 0.2], [1, 0.4], [2, 0.4]], 2], [4, 1]],
                      [[0, 4], [3, 3]],
                      []]
 
    my_new_v_list = []
    # my_new_v_list = next_v(my_pi, my_r_list, my_old_v_list, my_weight_list)
    for i in range(100):
        my_new_v_list = next_v(my_pi, my_r_list, my_old_v_list, my_weight_list)
        my_old_v_list = my_new_v_list
 
    print(my_new_v_list)

[6.0, 8.0, 10.0, 6.0, 0.0]
